<a href="https://colab.research.google.com/github/dellacortelab/nhanes_gigl/blob/main/Dietary_GI_GL_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's learn how to add Dietary GI variables to a subset of the NHANES national health survey dataset.

1. We will load the dataset.
2. We will assign GI values.
3. We calculate GL values.
4. We aggregate the data get dietary GL values.
5. We calculate dietary GI values.
6. We perform some basic reporting.

In [ ]:
!git clone https://github.com/dellacortelab/nhanes_gigl.git

In [ ]:
import pickle
with open('./nhanes_gigl/demo_data.pkl', 'rb') as f:
  df = pickle.load(f)

Explore the Data Frame.

In [ ]:
df

Let's extract the unique food codes.

In [ ]:
food_codes = df['USDA food code'].unique().astype(int)
print('Example food code 1:', food_codes[0],'\nExmaple food code 2:', food_codes[1])


Now see if you can find the description of the food on: https://fdc.nal.usda.gov/

Now let's find the GI values associated with these food: https://glycemicindex.com/gi-search/

Imagine you would have to do this for over 10k food codes by hand! Enter the AI!

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from scipy.spatial.distance import cosine

# Initialize the BERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# Define the food descriptions
food1 = "Food Description from FoodData Central"
food2 = "Food Description from GI Database"

# Tokenize and encode the food descriptions
inputs1 = tokenizer(food1, return_tensors='pt', truncation=True, padding=True)
inputs2 = tokenizer(food2, return_tensors='pt', truncation=True, padding=True)

# Get the embeddings for the food descriptions
with torch.no_grad():
    embed1 = model(**inputs1).last_hidden_state.mean(dim=1)
    embed2 = model(**inputs2).last_hidden_state.mean(dim=1)

# Calculate the cosine similarity
similarity = 1 - cosine(embed1.numpy()[0,:], embed2.numpy()[0,:])

print(f"The cosine similarity between the food descriptions is {similarity}")

The code above could be used to compare thousands of food descriptions between different databases. It uses a somewhat aged AI, called BERT. Our paper uses a more modern Large Language Model from Open-AI (the makers of ChatGPT). But for our purposes this example suffices.

Let's assume that we used the AI to align all food codes with corresponding GI values. Here, we will take random numbers for sake of time.

In [ ]:
import numpy as np
gi_values = [np.random.randint(45,100) for food_code in food_codes]
food_codes_to_gi = dict(zip(food_codes, gi_values))
for key in list(food_codes_to_gi.keys())[:5]:
  print(key,':', food_codes_to_gi[key])

Now, let's add the new GI variable to our dataframe!

In [ ]:
df['GI'] = df['USDA food code'].map(food_codes_to_gi)
df

Now let's calculate the GL value for each meal. This needs to take into account the available carbohydrates.

In [ ]:
df['Available Carbohydrate'] = df['Carbohydrate (gm)'] - df['Dietary fiber (gm)']
df['GL'] = df['GI'] * df['Available Carbohydrate'] / 100
df

Now we can sum up all the food items consumed by a given respondent in the NHANES survey. If we do this for each day of reporting, we will get Dietary GL.

In [ ]:
agg_dict = {
    'Dietary 20 Year Weight': 'first',
    'Energy (kcal)': 'sum',
    'Data release cycle': 'first',
    'Available Carbohydrate':'sum',
    'GI': 'sum',
    'GL': 'sum'
}
summed_df = df.groupby(['respondent sequence number','Day']).agg(agg_dict).reset_index()
summed_df.rename(columns={'GL': 'Dietary GL'}, inplace=True)
summed_df['Dietary GI'] = summed_df['Dietary GL'] / summed_df['Available Carbohydrate']*100
summed_df


Finally, we need to average over the different days, for those who respondent more than once.

In [ ]:
agg_dict = {
    'Dietary 20 Year Weight': 'mean',
    'Energy (kcal)': 'mean',
    'Available Carbohydrate':'mean',
    'Dietary GI': 'mean',
    'Dietary GL': 'mean',
    'Data release cycle': 'first',

}
summed_df = summed_df.groupby(['respondent sequence number']).agg(agg_dict).reset_index()
summed_df

Now we could do some exploration of the participants. In the example data, we only have Data release cycle as a variable. Others are available from NHANES and were used in the paper. Here, however, we just focus on time trends. Let's plot the average values and approximate the errors.

In [ ]:
# prompt: create a plot showing means of df['Dietary GL'] aggregated by column Data release cycle

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.pointplot(x='Data release cycle', y='Dietary GL', data=summed_df)
plt.xlabel('Data Release Cycle')
plt.ylabel('Dietary GL')
plt.title('Dietary GL by Data Release Cycle')
plt.show()

plt.figure(figsize=(10, 6))
sns.pointplot(x='Data release cycle', y='Dietary GI', data=summed_df)
plt.xlabel('Data Release Cycle')
plt.ylabel('Dietary GI')
plt.title('Dietary GI by Data Release Cycle')
plt.show()


Check out the plots. Do you see any interesting trends? Hopefully not, we drew random variables for GI!

Before these values could be interpreted, some important things need to be done.

1. The Dietary GL variable should be "adjusted" for total energy intake according to the residual method.
2. We also need to exclude unreasonbale consumption levels.
3. We want to use the NHANES weights to correctly calculate means.

In [ ]:
#now we need to energy adjust!
from sklearn.linear_model import LinearRegression
import pandas as pd

for var in ['Dietary GI','Dietary GL', 'Available Carbohydrate']:
    #index to avoid nan!

    idx_energy = summed_df['Energy (kcal)'].notna()
    idx = summed_df['Energy (kcal)'].notna() & summed_df[var].notna()

    # Separate the independent variable (X) and the dependent variable (Y)
    X, Y = summed_df['Energy (kcal)'][idx].to_numpy().reshape(-1, 1),summed_df[var][idx].to_numpy().reshape(-1, 1)

    # Create a linear regression model
    model = LinearRegression()

    # Fit the model to the data
    model.fit(X, Y)

    # Get the slope (m) and y-intercept (b)
    slope = model.coef_[0]
    intercept = model.intercept_

    # Predict Y values based on the regression line
    predicted_Y = model.predict(X)

    #Avoid missing or execssive consumption levels
    condition1 = summed_df['Energy (kcal)'] > 600
    condition2 = summed_df['Energy (kcal)'] < 5000

    baseline_df = summed_df[(condition1 & condition2)]

    mean_kcal = (baseline_df['Energy (kcal)'] * baseline_df['Dietary 20 Year Weight']).sum() / baseline_df['Dietary 20 Year Weight'].sum()
    print('Baseline Energy intake mean', mean_kcal)
    baseline = model.predict(np.array([mean_kcal]).reshape(-1,1))[0][0]
    print(var, baseline)
    # Add the new variable
    summed_df['Adjusted '+var] = pd.NA

    summed_df.loc[idx, 'Adjusted '+var] = (Y - predicted_Y) + baseline

summed_df


This dataset could be used to calculate linear regressions. However, NHANES follows a complex sample design, requiring more involved error calculations. Details can be found here: https://wwwn.cdc.gov/nchs/nhanes/tutorials/varianceestimation.aspx .
Unfortunately, Python does not provide the right implementation to do this calculation (and our example dataset omits the necessary PSU information), but in R, Strata, or SAS this can be easily done.

One last quick check we can do is a correlation analysis. For this we can scatter two variables.

In [ ]:
#define the variables of interest
x_var = 'Energy (kcal)'
y_var = 'Adjusted Dietary GL'

#plot
plt.scatter(summed_df[x_var], summed_df[y_var])
plt.xlabel(x_var)
plt.ylabel(y_var)
plt.title('Rough Correlation Analysis')
plt.show()

Which two variable do you expect to be highly correlated? Test a few combinations and check your intuition.